In [ ]:
import numpy as np
from typing import Callable, Tuple
import matplotlib.pyplot as plt

plt.style.use('ggplot')

Understanding Fundamental Physics
The simple pendulum, consisting of a mass attached to a fixed pivot point by a string or rod, is a cornerstone in the study of classical mechanics. It provides an elegant way to understand oscillatory motion, gravitational effects, energy conservation, and even more advanced topics like chaos theory when modified to a double pendulum. The equations of motion derived from this system serve as a starting point for more complicated systems in physics, engineering, and beyond.

In [ ]:
# Define the right-hand side of the system of ODEs
def pendulum_ODE(t: float, y: np.ndarray, g: float, L: float) -> np.ndarray:
    theta, omega = y
    return np.array([omega, -g/L * np.sin(theta)])

# Explicit Euler method
def explicit_euler(f: Callable, t: np.ndarray, y0: np.ndarray, *params) -> np.ndarray:
    y = np.zeros((len(t), len(y0)))
    y[0] = y0
    for i in range(len(t) - 1):
        h = t[i + 1] - t[i]
        y[i + 1] = y[i] + f(t[i], y[i], *params) * h
    return y

# Runge-Kutta 4th order method
def runge_kutta_4(f: Callable, t: np.ndarray, y0: np.ndarray, *params) -> np.ndarray:
    y = np.zeros((len(t), len(y0)))
    y[0] = y0
    for i in range(len(t) - 1):
        h = t[i + 1] - t[i]
        k1 = f(t[i], y[i], *params)
        k2 = f(t[i] + 0.5 * h, y[i] + 0.5 * h * k1, *params)
        k3 = f(t[i] + 0.5 * h, y[i] + 0.5 * h * k2, *params)
        k4 = f(t[i] + h, y[i] + h * k3, *params)
        y[i + 1] = y[i] + (h / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
    return y

# Parameters for the pendulum
g = 9.81  # acceleration due to gravity (m/s^2)
L = 1.0  # length of pendulum (m)

# Initial conditions [theta, omega] (angle in radians, angular velocity in radians/sec)
y0 = np.array([np.pi  / 4, 0.0])  # small initial angle, no initial angular velocity

# Time grid for integration
t = np.linspace(0, 4 * np.pi, 2000)

# Solve using Explicit Euler
y_euler = explicit_euler(pendulum_ODE, t, y0, g, L)

# Solve using Runge-Kutta 4
y_rk4 = runge_kutta_4(pendulum_ODE, t, y0, g, L)


In [ ]:
# Analytical solution for a simple pendulum with small angles
def analytical_solution(t: np.ndarray, theta0: float, omega0: float, g: float, L: float) -> np.ndarray:
    omega_t = np.sqrt(g / L)
    theta = theta0 * np.cos(omega_t * t) + (omega0 / omega_t) * np.sin(omega_t * t)
    omega = -theta0 * omega_t * np.sin(omega_t * t) + omega0 * np.cos(omega_t * t)
    return theta, omega

# Calculate the analytical solution
theta_analytical, omega_analytical = analytical_solution(t, y0[0], y0[1], g, L)

# Plot the results
plt.figure(figsize=(18, 6))

# Plot for Analytical Solution
plt.subplot(1, 3, 1)
plt.title("Analytical Solution")
plt.plot(t, theta_analytical, label=r"$\theta$ (angle in radians)")
plt.plot(t, omega_analytical, label=r"$\omega$ (angular velocity in rad/s)")
plt.xlabel("Time (s)")
plt.ylabel("State variables")
plt.legend()

# Plot for Explicit Euler
plt.subplot(1, 3, 2)
plt.title("Explicit Euler")
plt.plot(t, y_euler[:, 0], label=r"$\theta$ (angle in radians)")
plt.plot(t, y_euler[:, 1], label=r"$\omega$ (angular velocity in rad/s)")
plt.xlabel("Time (s)")
plt.ylabel("State variables")
plt.legend()

# Plot for Runge-Kutta 4th Order
plt.subplot(1, 3, 3)
plt.title("Runge-Kutta 4th Order")
plt.plot(t, y_rk4[:, 0], label=r"$\theta$ (angle in radians)")
plt.plot(t, y_rk4[:, 1], label=r"$\omega$ (angular velocity in rad/s)")
plt.xlabel("Time (s)")
plt.ylabel("State variables")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Create a single figure and axis object for the combined animation
fig, ax = plt.subplots(figsize=(8, 8))

# Function to plot all pendulums at a given time index on the same axis
def plot_all_pendulums(ax, theta_analytical, theta_euler, theta_rk4, L):
    ax.clear()
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal', 'box')
    
    # Plot Analytical Solution
    x_analytical = L * np.sin(theta_analytical)
    y_analytical = -L * np.cos(theta_analytical)
    ax.plot([0, x_analytical], [0, y_analytical], 'r-', label="Analytical")
    ax.plot(x_analytical, y_analytical, 'ro')
    
    # Plot Explicit Euler
    x_euler = L * np.sin(theta_euler)
    y_euler = -L * np.cos(theta_euler)
    ax.plot([0, x_euler], [0, y_euler], 'g-', label="Explicit Euler")
    ax.plot(x_euler, y_euler, 'go')
    
    # Plot Runge-Kutta 4
    x_rk4 = L * np.sin(theta_rk4)
    y_rk4 = -L * np.cos(theta_rk4)
    ax.plot([0, x_rk4], [0, y_rk4], 'b-', label="Runge-Kutta 4")
    ax.plot(x_rk4, y_rk4, 'bo')
    
    ax.legend()
    ax.set_title("Pendulum Movement")

# Initialize animation for the combined plot
def update_combined(frame):
    plot_all_pendulums(ax, theta_analytical[frame], y_euler[frame, 0], y_rk4[frame, 0], L)

# Create the animation
ani_combined = FuncAnimation(fig, update_combined, frames=len(t), repeat=True)

# Save the animation as a GIF
ani_combined.save("pendulum_animation_combined_2.gif", writer=PillowWriter(fps=20))

plt.close(fig)  # Close the plot



In [ ]:
# # Create a single figure and axis object for the combined animation
# fig, ax = plt.subplots(figsize=(7, 7))

# # Initialize the lines for each solution
# line_analytical, = ax.plot([], [], 'r-', label='Analytical Solution')
# line_euler, = ax.plot([], [], 'g-', label='Explicit Euler')
# line_rk4, = ax.plot([], [], 'b-', label='Runge-Kutta 4th Order')

# # Initialize points for each solution
# point_analytical, = ax.plot([], [], 'ro')
# point_euler, = ax.plot([], [], 'go')
# point_rk4, = ax.plot([], [], 'bo')

# # Setting axis properties
# ax.set_xlim(-1.5, 1.5)
# ax.set_ylim(-1.5, 1.5)
# ax.set_aspect('equal', 'box')
# ax.legend()

# # Initialize the animation function
# def update(frame):
#     ax.set_title(f"Time: {t[frame]:.2f} s")
    
#     # Update for analytical solution
#     x_analytical = L * np.sin(theta_analytical[frame])
#     y_analytical = -L * np.cos(theta_analytical[frame])
#     line_analytical.set_data([0, x_analytical], [0, y_analytical])
#     point_analytical.set_data(x_analytical, y_analytical)
    
#     # Update for Explicit Euler
#     x_euler = L * np.sin(y_euler[frame, 0])
#     y_euler_coord = -L * np.cos(y_euler[frame, 0])
#     line_euler.set_data([0, x_euler], [0, y_euler_coord])
#     point_euler.set_data(x_euler, y_euler_coord)
    
#     # Update for Runge-Kutta 4th Order
#     x_rk4 = L * np.sin(y_rk4[frame, 0])
#     y_rk4_coord = -L * np.cos(y_rk4[frame, 0])
#     line_rk4.set_data([0, x_rk4], [0, y_rk4_coord])
#     point_rk4.set_data(x_rk4, y_rk4_coord)
    
#     return line_analytical, line_euler, line_rk4, point_analytical, point_euler, point_rk4

# # Create the animation
# ani_combined = FuncAnimation(fig, update, frames=len(t), blit=True, repeat=False)

# # Save the animation as a GIF
# writer_combined = PillowWriter(fps=20)
# ani_combined.save("pendulum_animation_combined.gif", writer=writer_combined)

# plt.close(fig)  # Close the plot



In [ ]:
# from matplotlib.animation import FuncAnimation, PillowWriter

# # Function to plot the pendulum at a given time index
# def plot_pendulum(ax, theta, L, title):
#     ax.clear()
#     ax.set_xlim(-1.5, 1.5)
#     ax.set_ylim(-1.5, 1.5)
#     ax.set_aspect('equal', 'box')
#     x = L * np.sin(theta)
#     y = -L * np.cos(theta)
#     ax.plot([0, x], [0, y], 'r-')
#     ax.plot(x, y, 'bo')
#     ax.set_title(title)

# # Create figure and axis objects
# # fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# # Initialize animations
# def update(frame):
#     plot_pendulum(ax[0], theta_analytical[frame], L, "Analytical Solution")
#     plot_pendulum(ax[1], y_euler[frame, 0], L, "Explicit Euler")
#     plot_pendulum(ax[2], y_rk4[frame, 0], L, "Runge-Kutta 4th Order")

# # Create the animation
# ani = FuncAnimation(fig, update, frames=len(t), repeat=False)

# # Save the animation as a GIF
# writer = PillowWriter(fps=20)
# ani.save("pendulum_animation.gif", writer=writer)

# plt.close(fig)  # Close the plot

# # Return the path to the saved GIF

